# spaCy Basics

**spaCy** (https://spacy.io/) is an open-source Python library that parses and "understands" large volumes of text. Separate models are available that cater to specific languages (English, French, German, etc.).

In this section we'll install and setup spaCy to work with Python, and then introduce some concepts related to Natural Language Processing.

# Installation and Setup (Skip if using Google Colab)

Installation is a two-step process. First, install spaCy using either conda or pip. Next, download the specific model you want, based on language.<br> For more info visit https://spacy.io/usage/

### 1. From the command line or terminal:
> `conda install -c conda-forge spacy`
> <br>*or*<br>
> `pip install -U spacy`

> ### Alternatively you can create a virtual environment:
> `conda create -n spacyenv python=3 spacy=2`

### 2. Next, also from the command line (you must run this as admin or use sudo):

> `python -m spacy download en`

> ### If successful, you should see a message like:

> **`Linking successful`**<br>
> `    C:\Anaconda3\envs\spacyenv\lib\site-packages\en_core_web_sm -->`<br>
> `    C:\Anaconda3\envs\spacyenv\lib\site-packages\spacy\data\en`<br>
> ` `<br>
> `    You can now load the model via spacy.load('en')`


# Working with spaCy in Python

This is a typical set of instructions for importing and working with spaCy. Don't be surprised if this takes awhile - spaCy has a fairly large library to load:

In [6]:
# Import spaCy and load the language library
import spacy.cli
spacy.cli.download('de_core_news_sm')
nlp = spacy.load('en_core_web_sm')
nlp = spacy.load( 'de_core_news_sm')

# Create a Doc object
doc = nlp(r'Tesla is looking at buying U.S. startup for $6 million') #u for unicode 
#print(doc.text)
# Print each token separately
for token in doc:
    print(token.text, token.pos_, token.dep_)

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
Tesla X mo
is X uc
looking X ROOT
at X sb
buying X nk
U.S. X sb
startup ADJ pnc
for PROPN pnc
$ PROPN subtok
6 NUM nmc
million VERB ROOT


In [3]:
!python -m spacy download de_core_news_sm


     |████████████████████████████████| 11.1MB 599kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.1.0-cp36-none-any.whl size=11073065 sha256=37e63bcceb4aea2c8d37bfd16f6938dd9499d92c68d2a59f8562bfd6f7fdaf63
  Stored in directory: /tmp/pip-ephem-wheel-cache-bp1q_nws/wheels/b4/8b/5e/d2ce5d2756ca95de22f50f68299708009a4aafda2aea79c4e4
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


This doesn't look very user-friendly, but right away we see some interesting things happen:
1. Tesla is recognized to be a Proper Noun, not just a word at the start of a sentence
2. U.S. is kept together as one entity (we call this a 'token')

As we dive deeper into spaCy we'll see what each of these abbreviations mean and how they're derived. We'll also see how spaCy can interpret the last three tokens combined `$6 million` as referring to ***money***.

___
# spaCy Objects

After importing the spacy module in the cell above we loaded a **model** and named it `nlp`.<br>Next we created a **Doc** object by applying the model to our text, and named it `doc`.<br>spaCy also builds a companion **Vocab** object that we'll cover in later sections.<br>The **Doc** object that holds the processed text is our focus here.

___
# Pipeline
When we run `nlp`, our text enters a *processing pipeline* that first breaks down the text and then performs a series of operations to tag, parse and describe the data.   Image source: https://spacy.io/usage/spacy-101#pipelines

![Pipeline1](https://drive.google.com/uc?id=1olB3ILCQ5k3GsVSi3976HU_GTcm4Jkiu)

We can check to see what components currently live in the pipeline. In later sections we'll learn how to disable components and add new ones as needed.

In [9]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f30a837dba8>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f30a85e5228>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f30a85e5288>)]

['tagger', 'parser', 'ner']

___
## Tokenization
The first step in processing text is to split up all the component parts (words & punctuation) into "tokens". These tokens are annotated inside the Doc object to contain descriptive information. We'll go into much more detail on tokenization in an upcoming lecture. For now, let's look at another example:

In [13]:
doc2 = nlp(u"Tesla isn't   looking into startup's anymore.")
for token in doc2:
  print(token.text, token.pos_, token.dep_)

Tesla PROPN nsubj
is VERB ROOT
n't ADV neg
   SPACE 
looking VERB acomp
into ADP prep
startup NOUN pobj
's PART case
anymore ADV advmod
. PUNCT punct


Notice how `isn't` has been split into two tokens. spaCy recognizes both the root verb `is` and the negation attached to it. Notice also that both the extended whitespace and the period at the end of the sentence are assigned their own tokens.

It's important to note that even though `doc2` contains processed information about each token, it also retains the original text:

In [14]:
doc2

Tesla isn't   looking into startup's anymore.

In [15]:
doc2[0]

Tesla

In [16]:
type(doc2)

spacy.tokens.doc.Doc

___
## Part-of-Speech Tagging (POS)
The next step after splitting the text up into tokens is to assign parts of speech. In the above example, `Tesla` was recognized to be a ***proper noun***. Here some statistical modeling is required. For example, words that follow "the" are typically nouns.

For a full list of POS Tags visit https://spacy.io/api/annotation#pos-tagging

In [17]:
doc2[0].pos_

'PROPN'

___
## Dependencies
We also looked at the syntactic dependencies assigned to each token. `Tesla` is identified as an `nsubj` or the ***nominal subject*** of the sentence.

For a full list of Syntactic Dependencies visit https://spacy.io/api/annotation#dependency-parsing
<br>A good explanation of typed dependencies can be found [here](https://nlp.stanford.edu/software/dependencies_manual.pdf)

In [20]:
doc2[0].dep_

'nsubj'

To see the full name of a tag use `spacy.explain(tag)`

In [22]:
spacy.explain(u'nsubj')

'nominal subject'

In [67]:
print(doc2.root.text)

AttributeError: ignored

___
## Additional Token Attributes
We'll see these again in upcoming lectures. For now we just want to illustrate some of the other information that spaCy assigns to tokens:

|Tag|Description|doc2[0].tag|
|:------|:------:|:------|
|`.text`|The original word text<!-- .element: style="text-align:left;" -->|`Tesla`|
|`.lemma_`|The base form of the word|`tesla`|
|`.pos_`|The simple part-of-speech tag|`PROPN`/`proper noun`|
|`.tag_`|The detailed part-of-speech tag|`NNP`/`noun, proper singular`|
|`.shape_`|The word shape – capitalization, punctuation, digits|`Xxxxx`|
|`.is_alpha`|Is the token an alpha character?|`True`|
|`.is_stop`|Is the token part of a stop list, i.e. the most common words of the language?|`False`|

In [23]:
# Lemmas (the base form of the word):
print(doc2[4].text)
print(doc2[4].lemma_)

looking
look


In [35]:
# Simple Parts-of-Speech & Detailed Tags:
print(doc2[4].tag_)
print(doc2[4].text+": "+doc2[4].dep_)
spacy.explain(u'acomp')




VBG
looking: acomp


'adjectival complement'

In [27]:
# Word Shapes:
print(doc2[4].text+': '+doc2[4].shape_)
print(doc2[0].text+': '+doc2[0].shape_)

looking: xxxx
Tesla: Xxxxx


In [28]:
# Boolean Values:
print(doc2[0].is_alpha)
print(doc2[0].is_stop)

True
False


___
## Spans
Large Doc objects can be hard to work with at times. A **span** is a slice of Doc object in the form `Doc[start:stop]`.

In [0]:
doc3 = nlp(u'Although commmonly attributed to John Lennon from his song "Beautiful Boy", \
the phrase "Life is what happens to us while we are making other plans" was written by \
cartoonist Allen Saunders and published in Reader\'s Digest in 1957, when Lennon was 17.')

In [40]:
life_quote = doc3[16:30]
life_quote

"Life is what happens to us while we are making other plans"

In [38]:
type(life_quote)

spacy.tokens.span.Span

In upcoming lectures we'll see how to create Span objects using `Span()`. This will allow us to assign additional information to the Span.

___
## Sentences
Certain tokens inside a Doc object may also receive a "start of sentence" tag. While this doesn't immediately build a list of sentences, these tags enable the generation of sentence segments through `Doc.sents`. Later we'll write our own segmentation rules.

In [0]:
doc4 = nlp(u'This is the first sentence I.M. This is another sentence? This is the last sentence.')

In [61]:
for sent in doc4.sents:
  print(sent)

This is the first sentence I. M.
This is another sentence?
This is the last sentence.


In [70]:
for token in doc4:
  print(token.text+":"+token.head.text)

This:is
is:is
the:sentence
first:sentence
sentence:is
I.M.:sentence
This:is
is:is
another:sentence
sentence:is
?:is
This:is
is:is
the:sentence
last:sentence
sentence:is
.:is


In [68]:
#import spacy

#nlp = spacy.load("en_core_web_sm")
doc6 = nlp("Autonomous cars shift insurance liability toward manufacturers")
for token in doc6:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Autonomous amod cars NOUN []
cars nsubj shift VERB [Autonomous]
shift ROOT shift VERB [cars, liability]
insurance compound liability NOUN []
liability dobj shift VERB [insurance, toward]
toward prep liability NOUN [manufacturers]
manufacturers pobj toward ADP []


## Next up: Tokenization